In [2]:
from langchain.llms import GooglePalm
import os

from dotenv import load_dotenv
load_dotenv()

llm = GooglePalm(google_api_key=os.environ["GOOGLE_API_KEY"], temperature=0.1)

/home/delhivery/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [62]:
discussion = llm("how to start a logistic company")
print(discussion)

**Step 1: Write a business plan**

A business plan is a roadmap for your logistics company. It will help you define your goals, strategies, and financial projections. Your business plan should include the following sections:

* Executive summary
* Company description
* Market analysis
* Service offerings
* Marketing plan
* Financial plan
* Management team
* Appendix

**Step 2: Get the necessary permits and licenses**

The specific permits and licenses you need will vary depending on the state and city where you operate your business. Some common requirements include a business license, a tax ID number, and a permit to operate a warehouse.

**Step 3: Choose a legal structure**

There are three main legal structures for businesses: sole proprietorship, partnership, and corporation. The structure you choose will affect your taxes, liability, and other aspects of your business.

* Sole proprietorship: This is the simplest and least expensive structure. You are personally liable for all deb

In [3]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path="delhivery_faq.csv",source_column="prompt")

data = loader.load()

In [4]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-large")

e = instructor_embeddings.embed_query("I have not got my order. when can I expect that?")

load INSTRUCTOR_Transformer
max_seq_length  512


In [ ]:
e[:5]

[-0.012755748815834522,
 0.013954374939203262,
 -0.017030129209160805,
 0.03949020430445671,
 0.06731397658586502]

In [5]:
from langchain.vectorstores import FAISS

vectordb = FAISS.from_documents(documents=data,embedding=instructor_embeddings)

print(vectordb)

retriever = vectordb.as_retriever(score_threshold = 0.7)

In [64]:
rdocs = retriever.get_relevant_documents("I have not got my order. when can I expect that?")

rdocs


[Document(page_content="prompt: What do I do if tracking page doesn't show the expected date of arrival?\nresponse: Delhivery will be able to provide estimated date of arrival only after the shipment has been picked up from the merchant. In the meantime, please contact your merchant from where you placed the order for further assistance.", metadata={'source': "What do I do if tracking page doesn't show the expected date of arrival?", 'row': 3}),
 Document(page_content='prompt: How can I get the delivery of my shipment if I missed it today?\nresponse: Do not worry if you missed your order. Delhivery usually attempts to deliver the shipment on the following business day if you have missed a delivery.', metadata={'source': 'How can I get the delivery of my shipment if I missed it today?', 'row': 5}),
 Document(page_content='prompt: My shipment is out for delivery. When will I receive it?\nresponse: Your shipment can be delivered to you anytime between 10:00 am and 8:00 pm on the same day 

In [7]:
from langchain.prompts import PromptTemplate

prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know. Please raise a ticket for your query." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""

PROMPT = PromptTemplate(template=prompt_template,input_variables=['context','question']
            )

chain_type_kwargs = {"prompt": PROMPT, "verbose":True}

In [ ]:
from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(llm=llm,
                            chain_type="stuff",
                            retriever=retriever,
                            input_key="query",
                            return_source_documents=True,
                            verbose=True
                        )

In [8]:
from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(llm=llm,
                            chain_type="stuff",
                            retriever=retriever,
                            input_key="query",
                            return_source_documents=True,
                            verbose=True,
                            chain_type_kwargs=chain_type_kwargs
                        )

In [39]:
query = "Can i get my shipment tomorrow and please share the delivery boy number"
chain({"query": query})



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know. Please raise a ticket for your query." Don't try to make up an answer.

CONTEXT: prompt: How can I get the delivery of my shipment if I missed it today?
response: Do not worry if you missed your order. Delhivery usually attempts to deliver the shipment on the following business day if you have missed a delivery.

prompt: How can I get the contact number of my delivery partner?
response: As a part of the company policy, we do not share the phone numbers of our delivery partners. However, please be assured that our representatives will contact you if t

{'query': 'Can i get my shipment tomorrow and please share the delivery boy number',
 'result': 'Do not worry if you missed your order. Delhivery usually attempts to deliver the shipment on the following business day if you have missed a delivery. As a part of the company policy, we do not share the phone numbers of our delivery partners.',
 'source_documents': [Document(page_content='prompt: How can I get the delivery of my shipment if I missed it today?\nresponse: Do not worry if you missed your order. Delhivery usually attempts to deliver the shipment on the following business day if you have missed a delivery.', metadata={'source': 'How can I get the delivery of my shipment if I missed it today?', 'row': 5}),
  Document(page_content='prompt: How can I get the contact number of my delivery partner?\nresponse: As a part of the company policy, we do not share the phone numbers of our delivery partners. However, please be assured that our representatives will contact you if they face a

In [9]:
chain("does delhivery provide bfsi shipments?")



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know. Please raise a ticket for your query." Don't try to make up an answer.

CONTEXT: prompt: How can I get the delivery of my shipment if I missed it today?
response: Do not worry if you missed your order. Delhivery usually attempts to deliver the shipment on the following business day if you have missed a delivery.

prompt: Why can I see no update to the status of my shipment?
response: Delhivery tracking updates are accurate and real time. However, if due to unforeseen circumstances, your shipment has not moved in the last few days, you can raise a sup

{'query': 'does delhivery provide bfsi shipments?',
 'result': "I don't know. Please raise a ticket for your query.",
 'source_documents': [Document(page_content='prompt: How can I get the delivery of my shipment if I missed it today?\nresponse: Do not worry if you missed your order. Delhivery usually attempts to deliver the shipment on the following business day if you have missed a delivery.', metadata={'source': 'How can I get the delivery of my shipment if I missed it today?', 'row': 5}),
  Document(page_content='prompt: Why can I see no update to the status of my shipment?\nresponse: Delhivery tracking updates are accurate and real time. However, if due to unforeseen circumstances, your shipment has not moved in the last few days, you can raise a support ticket here.', metadata={'source': 'Why can I see no update to the status of my shipment?', 'row': 6}),
  Document(page_content="prompt: What do I do if tracking page doesn't show the expected date of arrival?\nresponse: Delhivery

In [31]:
chain("I have not got my order. when can I expect that and do you deliver at home?")

{'query': 'I have not got my order. when can I expect that and do you deliver at home?',
 'result': 'Your shipment can be delivered to you anytime between 10:00 am and 8:00 pm on the same day when it is marked as out for delivery. Yes, we deliver at home.',
 'source_documents': [Document(page_content='prompt: How can I get the delivery of my shipment if I missed it today?\nresponse: Do not worry if you missed your order. Delhivery usually attempts to deliver the shipment on the following business day if you have missed a delivery.', metadata={'source': 'How can I get the delivery of my shipment if I missed it today?', 'row': 5}),
  Document(page_content="prompt: What do I do if tracking page doesn't show the expected date of arrival?\nresponse: Delhivery will be able to provide estimated date of arrival only after the shipment has been picked up from the merchant. In the meantime, please contact your merchant from where you placed the order for further assistance.", metadata={'source':

In [68]:
chain("I have not got my order. when can I expect that and can you deliver it in my office?")

{'query': 'I have not got my order. when can I expect that and can you deliver it in my office?',
 'result': 'Your shipment can be delivered to you anytime between 10:00 am and 8:00 pm on the same day when it is marked as out for delivery. We can deliver your shipment to your office address if it is within our delivery area.',
 'source_documents': [Document(page_content='prompt: How can I get the delivery of my shipment if I missed it today?\nresponse: Do not worry if you missed your order. Delhivery usually attempts to deliver the shipment on the following business day if you have missed a delivery.', metadata={'source': 'How can I get the delivery of my shipment if I missed it today?', 'row': 5}),
  Document(page_content='prompt: My shipment is out for delivery. When will I receive it?\nresponse: Your shipment can be delivered to you anytime between 10:00 am and 8:00 pm on the same day when it is marked as out for delivery.', metadata={'source': 'My shipment is out for delivery. When

In [76]:
chain("Do you help in returns of product?")

{'query': 'Do you help in returns of product?',
 'result': 'Delhivery does not handle returns of products. Please contact the merchant from where you placed the order for further assistance.',
 'source_documents': [Document(page_content='prompt: How can I get the delivery of my shipment if I missed it today?\nresponse: Do not worry if you missed your order. Delhivery usually attempts to deliver the shipment on the following business day if you have missed a delivery.', metadata={'source': 'How can I get the delivery of my shipment if I missed it today?', 'row': 5}),
  Document(page_content='prompt: How can I get the contact number of my delivery partner?\nresponse: As a part of the company policy, we do not share the phone numbers of our delivery partners. However, please be assured that our representatives will contact you if they face any difficulty in locating your address.', metadata={'source': 'How can I get the contact number of my delivery partner?', 'row': 7}),
  Document(page_

In [21]:
chain("does delhivery provide open box delivery?")

{'query': 'does delhivery provide open box delivery?',
 'result': "I don't know. Please raise a ticket for your query.",
 'source_documents': [Document(page_content='prompt: How can I get the delivery of my shipment if I missed it today?\nresponse: Do not worry if you missed your order. Delhivery usually attempts to deliver the shipment on the following business day if you have missed a delivery.', metadata={'source': 'How can I get the delivery of my shipment if I missed it today?', 'row': 5}),
  Document(page_content="prompt: What do I do if tracking page doesn't show the expected date of arrival?\nresponse: Delhivery will be able to provide estimated date of arrival only after the shipment has been picked up from the merchant. In the meantime, please contact your merchant from where you placed the order for further assistance.", metadata={'source': "What do I do if tracking page doesn't show the expected date of arrival?", 'row': 3}),
  Document(page_content="prompt: How can I track